In [ ]:
import torch
import numpy as np
import pandas as pd
import os
import random
import pickle
import math

from sklift.metrics import qini_auc_score, uplift_auc_score
from sklift.viz import plot_qini_curve, plot_uplift_curve

torch.manual_seed(2)
torch.cuda.manual_seed(2)
torch.backends.cudnn.deterministic = True
np.random.seed(2)
random.seed(2)

In [ ]:
torch.cuda.set_device('cuda:3')
device = torch.device('cuda:3')

In [ ]:
IDs = ['subject_id', 'hadm_id', 'icustay_id']
features = ['age', 'gender', 'bmi', 'apsiii', 'sofa', 'smoker', 'hemoglobin']
treatment = 'oxygenation'
outcome = 'death90'

df = pd.read_csv('df_oxygenation.csv')

In [ ]:
cov = np.float32(np.load('matrix_temporal.npy'))
cov = cov / np.max(cov)

In [ ]:
df_final = df.copy()

df_final.head()

In [ ]:
df_final.head()
df_final.fillna(0)

In [ ]:
def load_data(X, y):
    X = torch.unsqueeze(X, 1)
    indices = torch.randperm(y.shape[0])
    X = X[indices]
    y = y[indices.numpy()]
    train_n = int(0.7 * len(y))
    return X[:train_n], y[:train_n], X[train_n:], y[train_n:]

In [ ]:
def make_pairs(arrays, labels):
    pair_arrays = []
    pair_labels = []
    
    idx = np.array(range(0, len(arrays)))
    
    for index in range(len(arrays)):
        current_array = arrays[index]
        label = labels[index]

        first_idx = np.random.choice(idx)
        first_array = arrays[first_idx]
        first_label = labels[first_idx]

        pair_arrays.append(torch.stack([current_array, first_array], dim=0))
        pair_labels.append([abs(label - first_label)])
        
        second_idx = np.random.choice(idx)
        second_array = arrays[second_idx]
        second_label = labels[second_idx]
        
        pair_arrays.append(torch.stack([current_array, second_array], dim=0))
        pair_labels.append([abs(label - second_label)])
        
        third_idx = np.random.choice(idx)
        third_array = arrays[third_idx]
        third_label = labels[third_idx]
        
        pair_arrays.append(torch.stack([current_array, third_array], dim=0))
        pair_labels.append([abs(label - third_label)])
    
    # return a 2-tuple of our patient pairs and labels
    return (torch.stack(pair_arrays), torch.Tensor(pair_labels))

In [ ]:
k = list(np.arange(1, 11, 1)) + list(np.arange(15, 105, 5))

MSE_table = pd.DataFrame(columns=k)
Qini_table = pd.DataFrame(columns=k)
AUUC_table = pd.DataFrame(columns=k)

In [ ]:
#Training patients with statin

df_final_db_1 = pd.read_csv('pats_db_1_balanced.csv')

In [ ]:
padded_array_1 = torch.load('pats_db_1_balanced.pt').cpu()

In [ ]:
padded_array_1[padded_array_1 != padded_array_1] = 0
torch.isnan(padded_array_1).any()

In [ ]:
padded_array_1.shape

In [ ]:
df_final_db_1.columns

In [ ]:
X_train, y_train, X_test, y_test = load_data(padded_array_1, df_final_db_1['propensity_score'].to_numpy())

(pair_train, label_train) = make_pairs(X_train, y_train)
(pair_test, label_test) = make_pairs(X_test, y_test)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

tensor_X_train_1 = pair_train[:, 0]
tensor_X_train_2 = pair_train[:, 1]
tensor_y_train = label_train[:]
train_dataset = TensorDataset(tensor_X_train_1, tensor_X_train_2, tensor_y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64)

tensor_X_test_1 = pair_test[:, 0]
tensor_X_test_2 = pair_test[:, 1]
tensor_y_test = label_test[:]
test_dataset = TensorDataset(tensor_X_test_1, tensor_X_test_2, tensor_y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [ ]:
import torch.nn as nn

class Net(nn.Module):

    def __init__(self, n_filter, filter1_length, filter2_length, filter3_length, n_visit, n_feature, covariance):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter1_length, n_feature), padding=0, stride=1)
        self.conv2 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter2_length, n_feature), padding=0, stride=1)
        self.conv3 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter3_length, n_feature), padding=0, stride=1)
        self.activ1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter1_length - 1) / 2), 0))
        self.maxpool2 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter2_length - 1) / 2), 0))
        self.maxpool3 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter3_length - 1) / 2), 0))
        self.sim_matrix = nn.Parameter(torch.randn((n_filter, n_filter)), requires_grad=True)
        self.covariance = nn.Parameter(torch.Tensor(covariance), requires_grad=False)
        self.dropout1 = nn.Dropout(0.2)
        self.linear1 = nn.Linear(int(2 * n_filter) + 2, 1)
        self.sigmoid1 = nn.Sigmoid()

    def forward_once(self, x):
        zeropad = nn.ZeroPad2d((0,0,1,0))
        output1 = self.conv1(x)
        output1 = self.activ1(output1)
        if (filter1_length % 2) == 0:
            output1 = zeropad(output1)
        output1 = self.maxpool1(output1)
        output1 = output1.view(-1, list(output1.size())[1])
        output2 = self.conv2(x)
        output2 = self.activ1(output2)
        if (filter2_length % 2) == 0:
            output2 = zeropad(output2)
        output2 = self.maxpool2(output2)
        output2 = output2.view(-1, list(output2.size())[1])
        output3 = self.conv3(x)
        output3 = self.activ1(output3)
        if (filter3_length % 2) == 0:
            output3 = zeropad(output3)
        output3 = self.maxpool3(output3)
        output3 = output3.view(-1, list(output3.size())[1])
        output = torch.cat((output1, output2, output3), 1)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        sim_temp = torch.matmul(output1, self.sim_matrix)
        sim_val1 = (sim_temp * output2).sum(dim=1)
        sim_val1 = sim_val1.view((-1, 1))
        mean_input1 = torch.mean(input1[:,:,:,:], 2)
        mean_input2 = torch.mean(input2[:,:,:,:], 2)
        delta = mean_input1 - mean_input2
        sim_val2 = torch.sqrt(torch.matmul(torch.matmul(delta, self.covariance.view(-1, self.covariance.size()[0], self.covariance.size()[1])), torch.transpose(delta, 1, 2))).view(-1, 1)
        output = torch.cat((output1, output2, sim_val1, sim_val2), 1)
        output = self.dropout1(output)
        output = self.linear1(output)
        output = self.sigmoid1(output)
        return output

In [ ]:
n_filter = 96
filter1_length = 8
filter2_length = 8
filter3_length = 8
n_visit = padded_array_1.shape[1]
n_feature = padded_array_1.shape[2]

net_1 = Net(n_filter, filter1_length, filter2_length, filter3_length, n_visit, n_feature, cov).to(device)

In [ ]:
from torchinfo import summary

summary(net_1, input_size=((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]), (X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3])))

In [ ]:
for name, param in net_1.named_parameters():
    if param.requires_grad:
        print(name, param.data, param.size())

In [ ]:
net_1.parameters

In [ ]:
print(list(net_1.parameters()))

In [ ]:
from torch import optim
import torchmetrics

# train the model
def train_NET(train_loader, val_loader, run_device, model, epochs=1000):
    counter = []
    train_loss_history = [] 
    iteration_number = 0
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # enumerate epochs
    for epoch in range(epochs):
        train_loss_total = 0
        train_steps = 0
        train_metric = torchmetrics.MeanSquaredError()
        # enumerate mini batches
        for i, (input1, input2, label) in enumerate(train_loader):
            # put data on gpu
            pat1 = input1.to(run_device)
            pat2 = input2.to(run_device)
            label = label.to(run_device)
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(pat1, pat2)
            # calculate loss
            train_loss = criterion(yhat, label)
            # calculate accuracy
            acc_train = train_metric(yhat.cpu(), label.cpu())
            # credit assignment
            train_loss.backward()
            # update model weights
            optimizer.step()
            # print statistics
            train_loss_total += train_loss.item()
            train_steps += 1
        print('Epoch number {}\n Current train loss {}\n Current train MSE {}\n'.format(epoch + 1, train_loss_total / train_steps, train_metric.compute()))
        iteration_number += 1
        counter.append(iteration_number)
        train_loss_history.append(train_loss_total / train_steps)
        
        # validation loss
        val_loss_total = 0
        val_steps = 0
        val_metric = torchmetrics.MeanSquaredError()
        for j, (input1, input2, label) in enumerate(val_loader):
            with torch.no_grad():
                # put data on gpu
                pat1 = input1.to(run_device)
                pat2 = input2.to(run_device)
                label = label.to(run_device)
                # compute the model output
                yhat = model(pat1, pat2)
                # calculate loss
                val_loss = criterion(yhat, label)
                # calculate accuracy
                acc_val = val_metric(yhat.cpu(), label.cpu())
                # print statistics
                val_loss_total += val_loss.item()
                val_steps += 1
        print('Epoch number {}\n Current val loss {}\n Current val MSE {}\n'.format(epoch + 1, val_loss_total / val_steps, val_metric.compute()))

In [ ]:
train_NET(train_dataloader, test_dataloader, device, net_1, 500)

In [ ]:
#Training patients without statin

df_final_db_0 = pd.read_csv('pats_db_0_balanced.csv')

In [ ]:
padded_array_0 = torch.load('pats_db_0_balanced.pt').cpu()

In [ ]:
padded_array_0[padded_array_0 != padded_array_0] = 0
torch.isnan(padded_array_0).any()

In [ ]:
padded_array_0.shape

In [ ]:
X_train, y_train, X_test, y_test = load_data(padded_array_0, df_final_db_0['propensity_score'].to_numpy())

(pair_train, label_train) = make_pairs(X_train, y_train)
(pair_test, label_test) = make_pairs(X_test, y_test)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

tensor_X_train_1 = pair_train[:, 0]
tensor_X_train_2 = pair_train[:, 1]
tensor_y_train = label_train[:]
train_dataset = TensorDataset(tensor_X_train_1, tensor_X_train_2, tensor_y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64)

tensor_X_test_1 = pair_test[:, 0]
tensor_X_test_2 = pair_test[:, 1]
tensor_y_test = label_test[:]
test_dataset = TensorDataset(tensor_X_test_1, tensor_X_test_2, tensor_y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [ ]:
import torch.nn as nn

class Net(nn.Module):

    def __init__(self, n_filter, filter1_length, filter2_length, filter3_length, n_visit, n_feature, covariance):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter1_length, n_feature), padding=0, stride=1)
        self.conv2 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter2_length, n_feature), padding=0, stride=1)
        self.conv3 = nn.Conv2d(1, int(n_filter/3), kernel_size=(filter3_length, n_feature), padding=0, stride=1)
        self.activ1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter1_length - 1) / 2), 0))
        self.maxpool2 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter2_length - 1) / 2), 0))
        self.maxpool3 = nn.MaxPool2d((n_visit, 1), stride=(n_visit, 1), padding=(int((filter3_length - 1) / 2), 0))
        self.sim_matrix = nn.Parameter(torch.randn((n_filter, n_filter)), requires_grad=True)
        self.covariance = nn.Parameter(torch.Tensor(covariance), requires_grad=False)
        self.dropout1 = nn.Dropout(0.2)
        self.linear1 = nn.Linear(int(2 * n_filter) + 2, 1)
        self.sigmoid1 = nn.Sigmoid()

    def forward_once(self, x):
        zeropad = nn.ZeroPad2d((0,0,1,0))
        output1 = self.conv1(x)
        output1 = self.activ1(output1)
        if (filter1_length % 2) == 0:
            output1 = zeropad(output1)
        output1 = self.maxpool1(output1)
        output1 = output1.view(-1, list(output1.size())[1])
        output2 = self.conv2(x)
        output2 = self.activ1(output2)
        if (filter2_length % 2) == 0:
            output2 = zeropad(output2)
        output2 = self.maxpool2(output2)
        output2 = output2.view(-1, list(output2.size())[1])
        output3 = self.conv3(x)
        output3 = self.activ1(output3)
        if (filter3_length % 2) == 0:
            output3 = zeropad(output3)
        output3 = self.maxpool3(output3)
        output3 = output3.view(-1, list(output3.size())[1])
        output = torch.cat((output1, output2, output3), 1)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        sim_temp = torch.matmul(output1, self.sim_matrix)
        sim_val1 = (sim_temp * output2).sum(dim=1)
        sim_val1 = sim_val1.view((-1, 1))
        mean_input1 = torch.mean(input1[:,:,:,:], 2)
        mean_input2 = torch.mean(input2[:,:,:,:], 2)
        delta = mean_input1 - mean_input2
        sim_val2 = torch.sqrt(torch.matmul(torch.matmul(delta, self.covariance.view(-1, self.covariance.size()[0], self.covariance.size()[1])), torch.transpose(delta, 1, 2))).view(-1, 1)
        output = torch.cat((output1, output2, sim_val1, sim_val2), 1)
        output = self.dropout1(output)
        output = self.linear1(output)
        output = self.sigmoid1(output)
        return output

In [ ]:
n_filter = 96
filter1_length = 8
filter2_length = 8
filter3_length = 8
n_visit = padded_array_0.shape[1]
n_feature = padded_array_0.shape[2]

net_0 = Net(n_filter, filter1_length, filter2_length, filter3_length, n_visit, n_feature, cov).to(device)

In [ ]:
from torchinfo import summary

summary(net_0, input_size=((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3]), (X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3])))

In [ ]:
for name, param in net_0.named_parameters():
    if param.requires_grad:
        print(name, param.data, param.size())

In [ ]:
net_0.parameters

In [ ]:
print(list(net_0.parameters()))

In [ ]:
from torch import optim
import torchmetrics

# train the model
def train_NET(train_loader, val_loader, run_device, model, epochs=1000):
    counter = []
    train_loss_history = [] 
    iteration_number = 0
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    # enumerate epochs
    for epoch in range(epochs):
        train_loss_total = 0
        train_steps = 0
        train_metric = torchmetrics.MeanSquaredError()
        # enumerate mini batches
        for i, (input1, input2, label) in enumerate(train_loader):
            # put data on gpu
            pat1 = input1.to(run_device)
            pat2 = input2.to(run_device)
            label = label.to(run_device)
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(pat1, pat2)
            # calculate loss
            train_loss = criterion(yhat, label)
            # calculate accuracy
            acc_train = train_metric(yhat.cpu(), label.cpu())
            # credit assignment
            train_loss.backward()
            # update model weights
            optimizer.step()
            # print statistics
            train_loss_total += train_loss.item()
            train_steps += 1
        print('Epoch number {}\n Current train loss {}\n Current train MSE {}\n'.format(epoch + 1, train_loss_total / train_steps, train_metric.compute()))
        iteration_number += 1
        counter.append(iteration_number)
        train_loss_history.append(train_loss_total / train_steps)
        
        # validation loss
        val_loss_total = 0
        val_steps = 0
        val_metric = torchmetrics.MeanSquaredError()
        for j, (input1, input2, label) in enumerate(val_loader):
            with torch.no_grad():
                # put data on gpu
                pat1 = input1.to(run_device)
                pat2 = input2.to(run_device)
                label = label.to(run_device)
                # compute the model output
                yhat = model(pat1, pat2)
                # calculate loss
                val_loss = criterion(yhat, label)
                # calculate accuracy
                acc_val = val_metric(yhat.cpu(), label.cpu())
                # print statistics
                val_loss_total += val_loss.item()
                val_steps += 1
        print('Epoch number {}\n Current val loss {}\n Current val MSE {}\n'.format(epoch + 1, val_loss_total / val_steps, val_metric.compute()))

In [ ]:
train_NET(train_dataloader, test_dataloader, device, net_0, 500)

In [ ]:
def load_data_for_eval(X):
    X = torch.unsqueeze(X, 1)
    
    return X[:]

In [ ]:
def compute_uplift(y_treat, y_untreat):
    uplift = np.average(y_treat) - np.average(y_untreat)
    
    return uplift

def calculate_mse(y_true, y_pred):
    MSE = np.average(np.square(np.array(y_true) - np.array(y_pred)))
    
    return MSE

In [ ]:
SEED = 0 # Repeat five times with different seed values

df_1visit = pd.read_csv('df_oxygenation.csv')

sample = pd.read_csv('sample.csv')
print(sample['icustay_id'].nunique())
sample = df_1visit[df_1visit['icustay_id'].isin(sample['icustay_id'].unique().tolist())].copy()
print(sample['icustay_id'].nunique())
sample_for_test_0 = sample[sample.death90 == 0].sample(n=50, random_state=SEED)
sample_for_test_1 = sample[sample.death90 == 1].sample(n=50, random_state=SEED, replace=True)
sample_for_test = sample_for_test_0.append(sample_for_test_1, ignore_index=True)

In [ ]:
import time
import torch.nn.functional as F

start = time.time()

padded_array_sample = torch.zeros((1, 8, len(features))).to(device)

for i in sample_for_test['icustay_id']:
    temp_pat = torch.tensor(df_final[df_final.icustay_id == i].copy()[features].values.astype(np.float32)).to(device)
    temp_pat = F.pad(temp_pat, pad=(0, 0, 0, 8 - temp_pat.shape[0]))
    temp_pat = torch.unsqueeze(temp_pat, 0)
    padded_array_sample = torch.vstack((padded_array_sample, temp_pat))
    
padded_array_sample = padded_array_sample[1:,:,:].type(torch.float32)

end = time.time()
print(end - start)

padded_array_sample

In [ ]:
padded_array_sample[padded_array_sample != padded_array_sample] = 0
torch.isnan(padded_array_sample).any()

In [ ]:
X_sample = load_data_for_eval(padded_array_sample)

In [ ]:
padded_array_db_1 = padded_array_1
padded_array_db_0 = padded_array_0

In [ ]:
padded_array_db_1[padded_array_db_1 != padded_array_db_1] = 0
print(torch.isnan(padded_array_db_1).any())

padded_array_db_0[padded_array_db_0 != padded_array_db_0] = 0
print(torch.isnan(padded_array_db_0).any())

In [ ]:
X_pats_db_1 = load_data_for_eval(padded_array_db_1)
X_pats_db_0 = load_data_for_eval(padded_array_db_0)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

tensor_X_sample = X_sample
sample_dataset = TensorDataset(tensor_X_sample)
sample_dataloader = DataLoader(sample_dataset, batch_size=1)

tensor_X_db_1 = X_pats_db_1
db_dataset_1 = TensorDataset(tensor_X_db_1)
db_dataloader_1 = DataLoader(db_dataset_1, batch_size=1000)

tensor_X_db_0 = X_pats_db_0
db_dataset_0 = TensorDataset(tensor_X_db_0)
db_dataloader_0 = DataLoader(db_dataset_0, batch_size=1000)

In [ ]:
MSE = []
Qini = []
AUUC = []

for no in k:
    sample_trans_outcome = []
    sample_outcome = []
    sample_oxygenation_status = []
    sample_uplift = []
    
    for i, (sample_input, oxygenation_status, true_outcome, trans_outcome) in enumerate(zip(sample_dataloader, sample_for_test[treatment], sample_for_test[outcome], sample_for_test['TransformedOutcome'])):
        sim_values_1 = []
        sim_values_0 = []
    
        db_pats_sim_values_1 = df_final_db_1.copy()
        db_pats_sim_values_0 = df_final_db_0.copy()
    
        for j, pat_db_1 in enumerate(db_dataloader_1):             
            with torch.no_grad():
            
                # put data on gpu
                pat1 = sample_input[0].to(device)
                pat2_1 = pat_db_1[0].to(device)
                pat1 = pat1.repeat(pat2_1.size()[0], 1, 1, 1)
                # compute the model output, convert to list and add to dictionary
                yhat_1 = net_1(pat1, pat2_1).cpu().squeeze().tolist()
                sim_values_1 = sim_values_1 + yhat_1
            
        sim_values_series_1 = pd.Series(sim_values_1, index=db_pats_sim_values_1.index)    
        db_pats_sim_values_1['sim_score'] = sim_values_series_1
        ordered_patients_1 = db_pats_sim_values_1.sort_values(by = 'sim_score', ascending=True).head(1000)
        
        for j, (pat_db_0) in enumerate(db_dataloader_0):             
            with torch.no_grad():
                
                # put data on gpu
                pat1 = sample_input[0].to(device)
                pat2_0 = pat_db_0[0].to(device)
                pat1 = pat1.repeat(pat2_0.size()[0], 1, 1, 1)
                # compute the model output, convert to list and add to dictionary
                yhat_0 = net_0(pat1, pat2_0).cpu().squeeze().tolist()
                sim_values_0 = sim_values_0 + yhat_0
    
        sim_values_series_0 = pd.Series(sim_values_0, index=db_pats_sim_values_0.index)    
        db_pats_sim_values_0['sim_score'] = sim_values_series_0
        ordered_patients_0 = db_pats_sim_values_0.sort_values(by = 'sim_score', ascending=True).head(1000)
        
        sample_trans_outcome.append(trans_outcome)
        sample_outcome.append(true_outcome)
        sample_oxygenation_status.append(oxygenation_status)
        
        ordered_patients_1_k = ordered_patients_1.head(no)
        ordered_patients_0_k = ordered_patients_0.head(no)
        
        y_1_k = ordered_patients_1_k[outcome].values
        y_0_k = ordered_patients_0_k[outcome].values
        
        uplift_k = compute_uplift(y_1_k, y_0_k)
        sample_uplift.append(uplift_k)
        
    MSE.append(calculate_mse(sample_trans_outcome, sample_uplift))
    Qini.append(qini_auc_score(sample_outcome, sample_uplift, sample_oxygenation_status))
    AUUC.append(uplift_auc_score(sample_outcome, sample_uplift, sample_oxygenation_status))

In [ ]:
series = pd.Series(MSE, index = MSE_table.columns)
MSE_table = MSE_table.append(series, ignore_index=True)
MSE_table

In [ ]:
series = pd.Series(Qini, index = Qini_table.columns)
Qini_table = Qini_table.append(series, ignore_index=True)
Qini_table

In [ ]:
series = pd.Series(AUUC, index = AUUC_table.columns)
AUUC_table = AUUC_table.append(series, ignore_index=True)
AUUC_table